In [7]:
import pandas as pd
import os
import numpy as np

In [6]:
#train 폴더와 test 폴더에 있는 positive, negative comments들과 그 라벨을 df에 append한다. (positive = 1, negative = 0)
basepath = 'aclImdb'

labels = {'pos':1, 'neg':0}
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos','neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)

df.columns = ['review','sentiment']

In [11]:
#df엔 양성과 음성이 순서대로 나열되어 있으므로 섞어줌
df = df.iloc[np.random.permutation(df.index)].reset_index(drop=True)

In [13]:
#csv파일로 df를 저장
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

In [29]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>','',str(text)) # HTML 태그들을 삭제
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|p)', text) #이모티콘들은 감정분석에 도움이 되기 때문에 삭제하지 않음
    text = (re.sub('[W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')) #문자나 숫자가 아닌 것을 모두 삭제 후, 이모티콘을 붙임
    return text

In [30]:
df['review'] = df['review'].apply(preprocessor)

In [32]:
#단어를 변하지 않는 기본 형태인 어간으로 바꾸는 어간 추출(stemming), makes -> make
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
tokenizer_porter('what doesn\'t kill you makes you stronger')

['what', "doesn't", 'kill', 'you', 'make', 'you', 'stronger']

In [ ]:
#불용어 제거, 불용어는 아주 흔하게 등장하는 단어로 문서의 종류를 구별하는데 별 도움이 안되는 단어들임 (is, and, has ..)
#tf idf vectorizer 에선, 특정한 단어가 많은 문서에서 등장할수록 그 단어의 가중치가 이미 낮아져 있음(term frequency inverse document frequency)
#따라서 불용어 제거는 tf-idf보다 기본 단어 빈도나 정규화된 단어 빈도를 사용할 때 더 도움됨.